## Oracle Cloud Instance POC


In [3]:
import os
import json
from pathlib import Path
from typing import Dict
from dataclasses import dataclass
from copy import deepcopy

import bravado
from bravado.client import SwaggerClient, CallableOperation
# from bravado.requests_client import RequestsClient
from gc3_query.lib.gc3_bravado.requests_client import OCRequestsClient
from bravado.requests_client import RequestsResponseAdapter
from bravado.swagger_model import load_file
from bravado_core.exception import MatchingResponseNotFound
from bravado.exception import HTTPBadRequest
from bravado.http_future import HttpFuture

from tinydb import TinyDB, Query

# from prettyprinter import pprint, pformat
from pprint import pprint, pformat

## https://medium.com/@betz.mark/validate-json-models-with-swagger-and-bravado-5fad6b21a825
# Validate json models with swagger and bravado
from bravado_core.spec import Spec

In [4]:
### B01
import toml
from pathlib import Path

from gc3_query.lib import *
from gc3_query.lib import BASE_DIR
from gc3_query.lib import List, Optional, Any, Callable, Dict, Tuple, Union, Set, Generator, Path

from gc3_query.lib.gc3_config import GC3Config, IDMCredential
from gc3_query.lib.iaas_classic.iaas_requests_http_client import IaaSRequestsHTTPClient
from gc3_query.lib.iaas_classic import IaaSServiceBase, API_SPECS_DIR, IaaSRequestsHTTPClient
from gc3_query.lib.iaas_classic.instances import Instances



In [5]:
### B01
ORACLE_VPN_CONNECTED=False
ASDF='ASDF'

In [6]:
### B01
service = 'Instances'
idm_domain = 'gc30003'
gc3_config = GC3Config()
service_cfg = gc3_config.iaas_classic.services[service]
idm_cfg = gc3_config.idm.domains[idm_domain]
http_client: IaaSRequestsHTTPClient = IaaSRequestsHTTPClient(idm_cfg=idm_cfg)

In [7]:
# from secrets import opc_username, opc_password
import keyring

opc_username = "eric.harris@oracle.com"
opc_password = keyring.get_password("gc3", "gc30003")
print(f"opc_username={opc_username}, opc_password={opc_password}")

opc_username=eric.harris@oracle.com, opc_password=None


In [8]:
### B01
opc_password = keyring.get_password(service_name="gc3@gc30003", username="eric.harris@oracle.com")
print(f"opc_username={opc_username}, opc_password={opc_password}")

opc_username=eric.harris@oracle.com, opc_password=V@nadium123!


In [9]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')


### Username/pass setup
idm_domain_username = f'/Compute-{idm_domain_name}/{opc_username}'
idm_service_instance_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idm_service_instance_username
# basic_auth_cred = _basic_auth_str(username, opc_password)


iaas_rest_endpoint: https://compute.uscom-central-1.oraclecloud.com
iaas_auth_endpoint: https://compute.uscom-central-1.oraclecloud.com/authenticate/



In [10]:
json_data = {"user": username, "password": opc_password}
print(f'idm_domain_username: {idm_domain_username}')
print(f'idm_service_instance_username: {idm_service_instance_username}')
print(f'username: {username}')

idm_domain_username: /Compute-gc30003/eric.harris@oracle.com
idm_service_instance_username: /Compute-587626604/eric.harris@oracle.com
username: /Compute-587626604/eric.harris@oracle.com


In [11]:
proxies = {
  'http': 'http://www-proxy-ash7.us.oracle.com:80',
  'https': 'https://www-proxy-ash7.us.oracle.com:80',
}

#### Create DB to store Instance data

In [12]:
instances_db = TinyDB('instances.tdb.json')

####  Authenticate and get header token


In [13]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+directory+json'),
                ])
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)


# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
requests_client.session.headers.update(headers)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")

response = requests_client.session.post(url=iaas_auth_endpoint, json=json_data)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
#     print(f"\nSession Cookies: {requests_client.session.cookies}")
#     print(f"\nResponse Headers['Set-Cookie']: {response.headers['Set-Cookie']}")
else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

# print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
cookie_header = {'Cookie': response.headers['Set-Cookie']}
# print(f"cookie_header: {cookie_header}\n")
requests_client.session.headers.update(cookie_header)
# print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")


Response OK: True, Status Code: 204, URL: https://compute.uscom-central-1.oraclecloud.com/authenticate/
Auth request succeess.



In [14]:
## Update the swagger spec to use https
spec_file_path = Path().joinpath('open_api_definitions/iaas_instances.json').resolve()
spec_dict = load_file(spec_file_path)
spec_dict['schemes'] = ['https']


In [15]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, application/oracle-compute-v3+directory+json, json, text/html')
                ])

# headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
#                 ('Accept', 'application/oracle-compute-v3+json, json, text/html')])

In [16]:
requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)
# requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         config={'also_return_response': True,
                                                 'validate_responses': True,
                                                 'validate_requests': True,
                                                 'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
#                                          config={'also_return_response': True,
#                                                  'validate_responses': False,
#                                                  'validate_requests': False,
#                                                  'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [17]:
for operation_name in dir(instances_resource):
    callable_operation = getattr(instances_resource, operation_name)
    print(f"{callable_operation.operation.operation_id}\t\t http_method={callable_operation.operation.http_method}\tpath={callable_operation.operation.path_name}")
    
    

deleteInstance		 http_method=delete	path=/instance/{name}
discoverInstance		 http_method=get	path=/instance/{container}
discoverRootInstance		 http_method=get	path=/instance/
getInstance		 http_method=get	path=/instance/{name}
listInstance		 http_method=get	path=/instance/{container}/
updateInstance		 http_method=put	path=/instance/{name}


In [18]:
callable_operation

In [19]:
@dataclass
class GetOpReturn:
    operation_details: Dict[str, Any]
    operation_response: RequestsResponseAdapter


def get_op(operation_id: str, **wkargs) -> Tuple[Dict[str, Any], RequestsResponseAdapter]:
    print(f"Operation {operation_id} starting.")
    try:
        operation_future = getattr(instances_resource, operation_id)(**wkargs)
        url = operation_future.future.request.url
        print(f"REST url for {operation_id}: {url}")
        operation_result, operation_response = operation_future.result()
    except HTTPBadRequest:
        print("Request failed for {operation_id}! ")
        print(f"URL: {operation_future.future.request.url}")
        raise
    operation_details = json.loads(operation_result)
    return GetOpReturn(operation_details,operation_response )
#     print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
#     print(f"Operation {operation_id} finished.\n")


In [20]:
operation_id = "discoverRootInstance"
print(f"Operation {operation_id} starting.")
try:
    operation_future = getattr(instances_resource, operation_id)()
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverRootInstance starting.
REST url for discoverRootInstance: https://compute.uscom-central-1.oraclecloud.com/instance/

discoverRootInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/:
 {'result': ['/Compute-587626604/']}

Operation discoverRootInstance finished.



In [21]:
### B01

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
http_future = instances.bravado_service_operations.discoverRootInstance()
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code





In [22]:
### B01
service_response

In [23]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")









operation_id=discoverRootInstance
service_response_status_code=200
http_response=200 OK
b01_result={"result": ["/Compute-587626604/"]}
s01_operation_result={"result": ["/Compute-587626604/"]}
is_same=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [24]:
## discoverInstance
## container="Compute-587626604/"
operation_id = "discoverInstance"

print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
    container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/"
    container = "Compute-587626604/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverInstance starting.
container: Compute-587626604/
REST url for discoverInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/

discoverInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/:
 {'result': ['/Compute-587626604/dhiru.vallabhbhai@oracle.com/',
            '/Compute-587626604/eric.harris@oracle.com/',
            '/Compute-587626604/manjunath.udupa@oracle.com/',
            '/Compute-587626604/mayurnath.gokare@oracle.com/',
            '/Compute-587626604/ramesh.dadhania@oracle.com/',
            '/Compute-587626604/seetharaman.nandyal@oracle.com/',
            '/Compute-587626604/siva.subramani@oracle.com/',
            '/Compute-587626604/tarek.orfaly@oracle.com/']}

Operation discoverInstance finished.



In [25]:
## discoverInstance
## container="Compute-587626604/eric.harris@oracle.com/"

operation_id = "discoverInstance"
print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
#     container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/{opc_username}/"
    container="Compute-587626604/eric.harris@oracle.com/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverInstance starting.
container: Compute-587626604/eric.harris@oracle.com/
REST url for discoverInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/

discoverInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/:
 {'result': ['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/',
            '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-PSFTHR92/',
            '/Compute-587626604/eric.harris@oracle.com/dbaas/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-cdmt-d07-lws/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d03-lws/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-02-essbase-bi/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-essbase-bi/',
            '/Compute-587626604/eric.harris@oracle.com/gc3-

In [26]:
### B01
## discoverInstance
## container="Compute-587626604/eric.harris@oracle.com/"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_container=instances.idm_user_container_name

print(f"b01_container={b01_container}")
print(f"s01_container={container}")
is_same_container = b01_container==container

print(f"is_same_container={is_same_container}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

b01_container=Compute-587626604/eric.harris@oracle.com/
s01_container=Compute-587626604/eric.harris@oracle.com/
is_same_container=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [27]:
### B01

http_future = instances.bravado_service_operations.discoverInstance(container=b01_container)
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [28]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")





operation_id=discoverInstance
service_response_status_code=200
http_response=200 OK
b01_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/", "/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-PSFTHR92/", "/Compute-587626604/eric.harris@oracle.com/dbaas/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-cdmt-d07-lws/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d03-lws/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-02-essbase-bi/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d04-essbase-bi/", "/Compute-587626604/eric.harris@oracle.com/gc3-naac-soar-d05-lws/", "/Compute-587626604/eric.harris@oracle.com/gc3_naac_soar_ebs1226_demo_01/", "/Compute-587626604/eric.harris@oracle.com/paas/"]}
s01_operation_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/", "/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-PSFTHR92/", "/Compute-587626604/eric.harris@oracle.com/dbaas/", "/Compu

In [29]:
operation_id = "listInstance"
print(f"Operation {operation_id} starting.")
try:
    # container = operation_details['result'][0].lstrip('/').rstrip('/')
    # container = 'Compute-587626604/eric.harris@oracle.com'
    container = 'Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

Operation listInstance starting.
container: Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1
REST url for listInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/

listInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/:
 {'result': ['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7']}

Operation listInstance finished.


In [30]:
operation_details

{'result': ['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7']}

In [31]:
operation_details['result']

['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7']

In [32]:
### B01
## listInstance
## container: "Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_container="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"

print(f"b01_container={b01_container}")
print(f"s01_container={container}")
is_same_container = b01_container==container

print(f"is_same_container={is_same_container}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

b01_container=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1
s01_container=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1
is_same_container=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [33]:
### B01

http_future = instances.bravado_service_operations.listInstance(container=b01_container)
request_url = http_future.future.request.url
service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [34]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")



operation_id=listInstance
service_response_status_code=200
http_response=200 OK
b01_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"]}
s01_operation_result={"result": ["/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"]}
is_same=True
ASDF=ASDF
ASDF=ASDF
ASDF=ASDF


In [35]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])


requests_client = OCRequestsClient()
requests_client.session.headers.update(headers)
if ORACLE_VPN_CONNECTED:
    requests_client.session.proxies.update(proxies)
# requests_client.session.proxies.update(proxies)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         # config={'also_return_response': True,
                                         #         'validate_responses': True,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         config={'also_return_response': True,
                                                 'validate_responses': False,
                                                 'validate_requests': False,
                                                 'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [36]:
operation_details_copy = deepcopy(operation_details)
print(operation_details_copy)

{'result': ['/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7']}


In [37]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details_copy['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")


Operation getInstance starting.
name: Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
REST url for getInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
Accept Header=[application/oracle-compute-v3+json, json, text/html], Content-Type Header=[application/oracle-compute-v3+json]

getInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7:
 {'account': '/Compute-587626604/default',
 'attributes': {'dns': {'domain': 'compute-587626604.oraclecloud.internal.',
                        'hostname': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.',
                        'nimbula_vcable-eth0': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.'},
                'netwo

In [38]:
operation_details

{'domain': 'compute-587626604.oraclecloud.internal.',
 'placement_requirements': ['/system/compute/placement/default',
  '/oracle/compute/dedicated/fcbaed9f-242b-42ce-ac77-dd727f9710eb',
  '/system/compute/allow_instances'],
 'ip': '10.19.0.246',
 'fingerprint': '34:a7:bb:00:67:50:85:f4:09:ef:8d:81:90:e7:46:72',
 'image_metadata_bag': '/oracle/machineimage_metadata/8921e067a70948379d279c9a372c9165',
 'site': '',
 'shape': 'oc4',
 'imagelist': None,
 'image_format': 'raw',
 'relationships': [],
 'availability_domain': '/uscom-central-1a',
 'networking': {'eth0': {'model': '',
   'seclists': ['/Compute-587626604/eric.harris@oracle.com/GC3NAACCDMT_PSFT'],
   'dns': ['gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.'],
   'vethernet': '/oracle/public/default',
   'nat': 'ipreservation:/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'}},
 'storage_attachments': [{'index': 1,
   'storage_volume_name': '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1_storage',


In [39]:
### B01
## getInstance
## name: "Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_name="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

print(f"b01_name={b01_name}")
print(f"s01_name={name}")
is_same_name = b01_name==name

print(f"is_same_name={is_same_name}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

b01_name=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
s01_name=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
is_same_name=True
ASDF=ASDF
ASDF=ASDF


In [40]:
### B01

http_future = instances.bravado_service_operations.getInstance(name=b01_name)
request_url = http_future.future.request.url
print(f"request_url={request_url}")
print(f"s01_url    ={request_url}")
url_same = url==request_url
print(f"url_same={url_same}")


request_url=https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
s01_url    =https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
url_same=True


In [41]:
### B01

service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [42]:
service_result = instances.bravado_service_operations.getInstance(name=b01_name).result()
service_result

('{"result": []}',
 <bravado.requests_client.RequestsResponseAdapter at 0x2186b5a54a8>)

```
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details_copy['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")
.
.
.
Operation getInstance starting.
name: Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
REST url for getInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
Accept Header=[application/oracle-compute-v3+json, json, text/html], Content-Type Header=[application/oracle-compute-v3+json]

getInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7:
 {'account': '/Compute-587626604/default',
 'attributes': {'dns': {'domain': 'compute-587626604.oraclecloud.internal.',
                        'hostname': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.',
                        'nimbula_vcable-eth0': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.'},
                'network': {'nimbula_vcable-eth0': {'address': ['c6:b0:6d:06:65:04',
                                                                '10.19.0.246'],
                                                    'dhcp_options': [],
                                                   
```
   
#### Accept Header=[application/oracle-compute-v3+json, json, text/html], Content-Type Header=[application/oracle-compute-v3+json]

   

In [43]:
instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_name="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

print(f"b01_name={b01_name}")
print(f"s01_name={name}")
is_same_name = b01_name==name

print(f"is_same_name={is_same_name}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

b01_name=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
s01_name=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
is_same_name=True
ASDF=ASDF
ASDF=ASDF


In [44]:
accept_headers = http_future.future.session.headers['Accept']
content_type_headers = http_future.future.session.headers['Content-Type']
print(f"accept_headers={accept_headers}")
print(f"content_type_headers={content_type_headers}")

accept_headers=application/oracle-compute-v3+directory+json
content_type_headers=application/oracle-compute-v3+json


In [45]:
### B01
## getInstance
## name: "Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
b01_name="Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7"

print(f"b01_name={b01_name}")
print(f"s01_name={name}")
is_same_name = b01_name==name

print(f"is_same_name={is_same_name}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")

b01_name=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
s01_name=Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
is_same_name=True
ASDF=ASDF
ASDF=ASDF


In [46]:
### B01
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])


http_future = instances.bravado_service_operations.getInstance(name=b01_name)
http_future.future.session.headers.update(headers)

request_url = http_future.future.request.url
print(f"request_url={request_url}")
print(f"s01_url    ={request_url}")
url_same = url==request_url
print(f"url_same={url_same}")


request_url=https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
s01_url    =https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
url_same=True


In [47]:
accept_headers = http_future.future.session.headers['Accept']
content_type_headers = http_future.future.session.headers['Content-Type']
print(f"accept_headers={accept_headers}")
print(f"content_type_headers={content_type_headers}")

accept_headers=application/oracle-compute-v3+json, json, text/html
content_type_headers=application/oracle-compute-v3+json


In [48]:
### B01

service_response = http_future.response()
result = service_response.result
http_response = service_response.incoming_response
http_response_json = http_response.json()
service_response_status_code = service_response.metadata.status_code

In [49]:
### B01
print(f"operation_id={operation_id}")
print(f"service_response_status_code={service_response_status_code}")
print(f"http_response={http_response}")
print(f"b01_result={result}")
# print(f"s01_operation_result={operation_result}")
is_same = result == operation_result
print(f"is_same={is_same}")



print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")
print(f"ASDF={ASDF}")


operation_id=getInstance
service_response_status_code=200
http_response=200 OK
b01_result={"domain": "compute-587626604.oraclecloud.internal.", "placement_requirements": ["/system/compute/placement/default", "/oracle/compute/dedicated/fcbaed9f-242b-42ce-ac77-dd727f9710eb", "/system/compute/allow_instances"], "ip": "10.19.0.246", "fingerprint": "34:a7:bb:00:67:50:85:f4:09:ef:8d:81:90:e7:46:72", "image_metadata_bag": "/oracle/machineimage_metadata/8921e067a70948379d279c9a372c9165", "site": "", "shape": "oc4", "imagelist": null, "image_format": "raw", "relationships": [], "availability_domain": "/uscom-central-1a", "networking": {"eth0": {"model": "", "seclists": ["/Compute-587626604/eric.harris@oracle.com/GC3NAACCDMT_PSFT"], "dns": ["gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal."], "vethernet": "/oracle/public/default", "nat": "ipreservation:/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"}}, "storage_attachments": [{"index": 1, "storage_volume_name": "/Compute-58

In [50]:
instances = Instances(service_cfg=service_cfg, idm_cfg=idm_cfg, http_client=http_client)
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])
http_future = instances.bravado_service_operations.getInstance(name=b01_name)
http_future.future.session.headers.update(headers)
service_result = instances.bravado_service_operations.getInstance(name=b01_name).result()
service_result

('{"domain": "compute-587626604.oraclecloud.internal.", "placement_requirements": ["/system/compute/placement/default", "/oracle/compute/dedicated/fcbaed9f-242b-42ce-ac77-dd727f9710eb", "/system/compute/allow_instances"], "ip": "10.19.0.246", "fingerprint": "34:a7:bb:00:67:50:85:f4:09:ef:8d:81:90:e7:46:72", "image_metadata_bag": "/oracle/machineimage_metadata/8921e067a70948379d279c9a372c9165", "site": "", "shape": "oc4", "imagelist": null, "image_format": "raw", "relationships": [], "availability_domain": "/uscom-central-1a", "networking": {"eth0": {"model": "", "seclists": ["/Compute-587626604/eric.harris@oracle.com/GC3NAACCDMT_PSFT"], "dns": ["gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal."], "vethernet": "/oracle/public/default", "nat": "ipreservation:/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1"}}, "storage_attachments": [{"index": 1, "storage_volume_name": "/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1_storage", "name": "/Compute-587626604/e

In [51]:
http_client.session.headers['Accept']

'application/oracle-compute-v3+json, json, text/html'

In [52]:
operation_details_copy = deepcopy(operation_details)
operation_details_copy

{'domain': 'compute-587626604.oraclecloud.internal.',
 'placement_requirements': ['/system/compute/placement/default',
  '/oracle/compute/dedicated/fcbaed9f-242b-42ce-ac77-dd727f9710eb',
  '/system/compute/allow_instances'],
 'ip': '10.19.0.246',
 'fingerprint': '34:a7:bb:00:67:50:85:f4:09:ef:8d:81:90:e7:46:72',
 'image_metadata_bag': '/oracle/machineimage_metadata/8921e067a70948379d279c9a372c9165',
 'site': '',
 'shape': 'oc4',
 'imagelist': None,
 'image_format': 'raw',
 'relationships': [],
 'availability_domain': '/uscom-central-1a',
 'networking': {'eth0': {'model': '',
   'seclists': ['/Compute-587626604/eric.harris@oracle.com/GC3NAACCDMT_PSFT'],
   'dns': ['gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.'],
   'vethernet': '/oracle/public/default',
   'nat': 'ipreservation:/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'}},
 'storage_attachments': [{'index': 1,
   'storage_volume_name': '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1_storage',


In [53]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details_copy['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
# print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")


Operation getInstance starting.


KeyError: 'result'

In [ ]:
type(operation_details)
print(operation_details.keys())

#### Insert results in to DB

In [ ]:
instances_db.insert(operation_details)

In [ ]:
instances_db.all()[-1]